In [546]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [547]:
import numpy as np
from timer import Timer
import pickle

In [548]:
neuronMax = 1
neuronMin = -1
neuronDtype = np.float32


class NeuralNetwork:
    @staticmethod
    def sigmoid(x, derivative=False):
        if derivative:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def relu(x, derivative=False):
        if derivative:
            return 1.0 * (x > 0)
        return np.maximum(0, x)

    @staticmethod
    def leakyRelu(x, derivative=False):
        if derivative:
            return 1.0 * (x > 0) + 0.01 * (x <= 0)
        return np.maximum(0.01 * x, x)

    @staticmethod
    def absolute(x, derivative=False):
        if derivative:
            return 1 if x > 0 else -1
        return np.abs(x)


    # @staticmethod
    # def softmax(x):
    #     exps = np.exp(x - np.max(x))
    #     return exps / np.sum(exps, axis=1, keepdims=True)

    # @staticmethod
    # def softmaxDerivative(x):
    #     return x * (1 - x)
    @staticmethod
    def mseLoss(errorVector, derivative=False):
        if derivative:
            return errorVector
        return np.mean(np.square(errorVector))*0.5

    def __init__(self, layerSizes, activationFunctions):
        self.layerSizes = layerSizes
        self.activationFunctions = activationFunctions
        self.layers = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i]).astype(neuronDtype)
            for i in range(len(layerSizes))
        ]
        self.weights = [
            np.random.uniform(
                low=neuronMin, high=neuronMax, size=layerSizes[i] * layerSizes[i + 1]
            ).astype(neuronDtype).reshape(layerSizes[i], layerSizes[i + 1])
            for i in range(len(layerSizes) - 1)
        ]
        self.biases = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i + 1]).astype(
                neuronDtype
            )
            for i in range(len(layerSizes) - 1)
        ]

    def loadInput(self, input):
        self.layers[0] = input.astype(neuronDtype)

    def readOutput(self):
        return self.layers[-1]

    def forward(self):
        for i in range(len(self.weights)):
            self.layers[i + 1] = self.activationFunctions[i](
                np.dot(self.layers[i], self.weights[i]) + self.biases[i]
            )



    def backpropagation(self, target, learning_rate, lossFunction):
        output_error = lossFunction(target - self.layers[-1], derivative=True)

        for i in range(len(self.layers) - 2, -1, -1):
            # Calculate the derivative of the activation function
            derivative = self.activationFunctions[i](self.layers[i + 1], derivative=True)

            # Calculate the error of the current layer
            layer_error = np.dot(output_error * derivative, self.weights[i].T) * learning_rate

            # Calculate the adjustments for the weights and biases
            weight_adjustments = np.outer(self.layers[i], output_error)
            bias_adjustments = output_error

            # Update the weights and biases
            self.weights[i] += weight_adjustments * learning_rate
            self.biases[i] += bias_adjustments * learning_rate

            # Set the output error for the next iteration
            # np.normalize(layer_error)
            output_error = layer_error


In [549]:
x = NeuralNetwork([5_000, 40, 40, 3], [NeuralNetwork.leakyRelu, NeuralNetwork.leakyRelu, NeuralNetwork.leakyRelu])
nnTimer = Timer(precision=3)
target = np.array([2, 3, 4])
input = np.random.uniform(low = 0, high = 1, size = 5_000)

In [550]:
x.loadInput(input)
x.forward()
print(x.readOutput())

[89.33454    55.644417   -0.56158143]


In [551]:
# learning_rate = 0.00001
# for i in range(200):
#     nnTimer.tic()
#     x.loadInput(input)
#     x.forward()
#     x.backpropagation(target, learning_rate)
#     print(x.readOutput())
#     nnTimer.toc()
# nnTimer.stats()
learning_rate = 0.00001
for i in range(200):
    nnTimer.tic()
    x.loadInput(input)
    x.forward()
    x.backpropagation(target, learning_rate, NeuralNetwork.mseLoss)
    print(x.readOutput(), NeuralNetwork.mseLoss(target - x.readOutput()))
    nnTimer.toc()
nnTimer.stats()

[89.33454    55.644417   -0.56158143] 1736.59412567521
[40.728992   26.345533   -0.53619474] 344.25430804065354
[19.174616   13.352728   -0.51094913] 70.41584446662348
[ 9.616208   7.590993  -0.4858441] 16.534439810618093
[ 5.3774467  5.035905  -0.4608788] 5.908582359243941
[ 3.4977589   3.9028394  -0.43605223] 3.7894933424687367
[ 2.6642175  3.4003737 -0.4113639] 3.343602567221851
[ 2.294536    3.1775608  -0.38681293] 3.2270678340233965
[ 2.1306233   3.078728   -0.36239862] 3.175630384538721
[ 2.057948   3.0349114 -0.3381201] 3.1373104693687544
[ 2.025679    3.0154867  -0.31397703] 3.1018828492607113
[ 2.0113888   3.0068653  -0.28996813] 3.067333903006208
[ 2.0050526   3.003043   -0.26609278] 3.0332637286688944
[ 2.002245    3.0013568  -0.24235028] 2.9995904635743176
[ 2.0009823   3.0006053  -0.21874008] 2.966294859785171
[ 2.000452   3.0002582 -0.1952611] 2.9333693362747244
[ 2.0002003   3.000117   -0.17191283] 2.9008094581182107
[ 2.000082    3.0000675  -0.14869456] 2.86861109381441

In [14]:
import numpy as np

size = 5
padding = 3

a = np.zeros((size + padding * 2, size + padding * 2))

for i in range(padding, size + padding):
    for j in range(padding, size + padding):
        a[i, j] = i - padding + (j - padding) * 10

In [15]:
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 10., 20., 30., 40.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., 11., 21., 31., 41.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  2., 12., 22., 32., 42.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  3., 13., 23., 33., 43.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4., 14., 24., 34., 44.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])